In [1]:
import random
from OCC.Display.WebGl.x3dom_renderer import *
from IPython.display import display, HTML
%matplotlib inline
caller_id = 0
def DisplayShape(shape,
                     vertex_shader=None,
                     fragment_shader=None,
                     export_edges=False,
                     color=(random.random(), random.random(), random.random()),
                     specular_color=(1, 1, 1),
                     shininess=0.9,
                     transparency=0.,
                     line_color=(0, 0., 0.),
                     line_width=2.,
                     mesh_quality=1.):
        def Show(src="<shape><appearance><material diffuseColor='0.603 0.894 0.909'></material></appearance> <box></box></shape>   ", height=400,  width=400):
            width=str(width)
            height=str(height)
            result = ""
            if(caller_id <2):
                result += " <script type='text/javascript' src='http://www.x3dom.org/download/x3dom.js'> </script>  <link rel='stylesheet' type='text/css' href='http://www.x3dom.org/download/x3dom.css'></link>"
           
            result +="<div style='height: "+height+"px;width: 100%;' id='x3dholder_"+str(caller_id)+"'  width='100%' height='"+height+"px'><x3d style='height: "+height+"px;width: 100%;' id='x3d"+str(caller_id)+"' width='100%' height='"+height+"px'><scene>"+src+"   </scene></x3d> </div>"
            return result
        x3d_exporter = X3DExporter(shape, vertex_shader, fragment_shader,
                                   export_edges, color,
                                   specular_color, shininess, transparency,
                                   line_color, line_width, mesh_quality)
        x3d_exporter.compute()
        tmp = x3d_exporter.to_x3dfile_string()
        temp_file_name = "tmp_"+str(++caller_id)+".x3d"
        if os.path.exists(temp_file_name): os.remove(temp_file_name)
        text_file = open(temp_file_name, "w")
        text_file.write(tmp)
        text_file.close()
        return HTML(Show("<inline url='./"+temp_file_name+"'> </inline> "))
    

import numpy
print(numpy.__path__)

['/opt/conda/lib/python3.5/site-packages/numpy']


In [47]:
import os
from OCC.StlAPI import StlAPI_Writer
from OCC.BRepAlgoAPI import BRepAlgoAPI_Cut
from OCC.BRepPrimAPI import BRepPrimAPI_MakeCylinder
from OCC.BRepMesh import BRepMesh_IncrementalMesh
from OCC.gp import gp_Pnt, gp_Ax2, gp_Dir, gp_Circ
from OCC.GeomAPI import GeomAPI_PointsToBSpline
from OCC.TColgp import TColgp_Array1OfPnt
from OCC.BRepBuilderAPI import BRepBuilderAPI_MakeEdge, BRepBuilderAPI_MakeWire, BRepBuilderAPI_MakeFace
from OCC.BRepOffsetAPI import BRepOffsetAPI_MakePipe
from OCC.BRepAlgoAPI import BRepAlgoAPI_Fuse
from OCC.BRepPrimAPI import BRepPrimAPI_MakeSphere

# first, create the shape
def pipe(point1, point2):
    makeWire = BRepBuilderAPI_MakeWire()
    edge = BRepBuilderAPI_MakeEdge(point1, point2).Edge()
    makeWire.Add(edge)
    makeWire.Build()
    wire = makeWire.Wire()

    dir = gp_Dir(point2.X() - point1.X(), point2.Y() - point1.Y(), point2.Z() - point1.Z())
    circle = gp_Circ(gp_Ax2(point1,dir), 0.2)
    profile_edge = BRepBuilderAPI_MakeEdge(circle).Edge()
    profile_wire = BRepBuilderAPI_MakeWire(profile_edge).Wire()
    profile_face = BRepBuilderAPI_MakeFace(profile_wire).Face()
    pipe = BRepOffsetAPI_MakePipe(wire, profile_face).Shape()
    return(pipe)

def sphere(centre, radius):
    sphere = BRepPrimAPI_MakeSphere (centre, radius).Shape()
    return(sphere)
#    display.DisplayShape(sphere, update=True)

if __name__ == '__main__':
    pipe1=pipe(gp_Pnt(0,0,0), gp_Pnt(0,0,1))
    sphere1=sphere(gp_Pnt(0,0,1), 0.2)
    pipe2=pipe(gp_Pnt(0,0,1), gp_Pnt(0,1,2))  
    sphere2=sphere(gp_Pnt(0,1,2), 0.2)
    pipe3=pipe(gp_Pnt(0,1,2), gp_Pnt(0,2,2))


glued1 = BRepAlgoAPI_Fuse(pipe1, sphere1).Shape()
glued2= BRepAlgoAPI_Fuse(glued1, pipe2).Shape()
glued3 = BRepAlgoAPI_Fuse(glued2, sphere2).Shape()
glued4 = BRepAlgoAPI_Fuse(glued3, pipe3).Shape()
geometry = DisplayShape(glued4)
display(geometry)

# then mesh it. This mesh is used by the STL exporter
# Note : if the mesh is not performed, the STL exporter
# won't do nothing
mesh = BRepMesh_IncrementalMesh(glued4, 0.6)
mesh.Perform()

stl_file = "./pipe_low_resolution.stl"
stl_exporter = StlAPI_Writer()
stl_exporter.SetASCIIMode(True)  # change to False if you need binary export
stl_exporter.Write(glued4, stl_file)


# then we change the mesh resolution
#mesh.SetDeflection(0.05)


from OCC.Display.SimpleGui import init_display
from OCC.TopoDS import TopoDS_Shape
from OCC.StlAPI import StlAPI_Reader

stl_reader = StlAPI_Reader()
fan_shp = TopoDS_Shape()
stl_reader.Read(fan_shp, stl_file)

exproted = DisplayShape(fan_shp)
display(exproted)

In [48]:
import subprocess
geoCounter = 0
def getGeo(stl_filename):
    geoFile = "Merge \""+stl_filename+"\";\nSurface Loop(1) = {1};\nVolume(1) = {1};\nPhysical Volume(1) = {1};\n"
    temp_file_name = "tmp_"+str(++geoCounter)+".geo"
    if os.path.exists(temp_file_name): os.remove(temp_file_name)
    text_file = open(temp_file_name, "w")
    text_file.write(geoFile)
    text_file.close()
    return temp_file_name;
out = ""
try:
    out = subprocess.check_output(
            ["gmsh", "gmsh -3 -algo meshadapt tmp_0.geo -o SFM.msh"],
            stderr=subprocess.STDOUT
            ).strip().decode('utf8')
except subprocess.CalledProcessError as e:
    out = e.output
    
getGeo(stl_file)
print(out)

b"Unexpected end of /proc/mounts line `overlay / overlay rw,relatime,lowerdir=/var/lib/docker/overlay2/l/KXAO4CMFO5EW4O2IZJRQIHCPFI:/var/lib/docker/overlay2/l/NASXZFW5UZSQSPRPWTFPSLGZSB:/var/lib/docker/overlay2/l/3ECOUWQRCKNRCEZG5GXQMRNLTF:/var/lib/docker/overlay2/l/MT2KF5SWYZ3KLOSGTZXDDDTXL6:/var/lib/docker/overlay2/l/EWEHL722SMA77OPCL7Z7PQNDA6:/var/lib/docker/overlay2/l/2TLN6ME67TLH32WHOCBDOTGBJ5:/var/lib/docker/overlay2/l/BKVVY5T4QQM5UXVSSPTXS5GRWW:/var/lib/docker/overlay2/l/TKQIIDSHVJFLRHEPHYY3CAAHAD:/var/lib/docker/overlay2/l/SO2KFZ3ZNW5BZ'\nUnexpected end of /proc/mounts line `6TNOMT7SWI4QT:/var/lib/docker/overlay2/l/KC24YTY2DUOXVESTMWBUNKEELF:/var/lib/docker/overlay2/l/5LZJR6HPC6SOVIDRDUD6FYPSWS:/var/lib/docker/overlay2/l/QJC6JUMCQMEGBE6GOTCCFF3GYJ:/var/lib/docker/overlay2/l/GGYWY3YMFNXLK4F25ZO3SFQZ35:/var/lib/docker/overlay2/l/PAPT7ON6DZMSQHSWBGYLYGWK6G:/var/lib/docker/overlay2/l/2W4UA2QEVRB2OJCHP5VPPOECF7:/var/lib/docker/overlay2/l/JMDH77HDS2KU5VT27MMBAVQHB2,upperdir=/var/lib/

In [49]:
!gmsh -3 -algo meshadapt tmp_0.geo -o SFM.msh
!dolfin-convert SFM.msh sfm.xml
!ls -la

Unexpected end of /proc/mounts line `overlay / overlay rw,relatime,lowerdir=/var/lib/docker/overlay2/l/KXAO4CMFO5EW4O2IZJRQIHCPFI:/var/lib/docker/overlay2/l/NASXZFW5UZSQSPRPWTFPSLGZSB:/var/lib/docker/overlay2/l/3ECOUWQRCKNRCEZG5GXQMRNLTF:/var/lib/docker/overlay2/l/MT2KF5SWYZ3KLOSGTZXDDDTXL6:/var/lib/docker/overlay2/l/EWEHL722SMA77OPCL7Z7PQNDA6:/var/lib/docker/overlay2/l/2TLN6ME67TLH32WHOCBDOTGBJ5:/var/lib/docker/overlay2/l/BKVVY5T4QQM5UXVSSPTXS5GRWW:/var/lib/docker/overlay2/l/TKQIIDSHVJFLRHEPHYY3CAAHAD:/var/lib/docker/overlay2/l/SO2KFZ3ZNW5BZ'
Unexpected end of /proc/mounts line `6TNOMT7SWI4QT:/var/lib/docker/overlay2/l/KC24YTY2DUOXVESTMWBUNKEELF:/var/lib/docker/overlay2/l/5LZJR6HPC6SOVIDRDUD6FYPSWS:/var/lib/docker/overlay2/l/QJC6JUMCQMEGBE6GOTCCFF3GYJ:/var/lib/docker/overlay2/l/GGYWY3YMFNXLK4F25ZO3SFQZ35:/var/lib/docker/overlay2/l/PAPT7ON6DZMSQHSWBGYLYGWK6G:/var/lib/docker/overlay2/l/2W4UA2QEVRB2OJCHP5VPPOECF7:/var/lib/docker/overlay2/l/JMDH77HDS2KU5VT27MMBAVQHB2,upperdir=/var/lib/doc

In [50]:
from dolfin import *
mesh = Mesh("./sfm.xml")
V = FunctionSpace(mesh, "CG", 1)

# Define variational problem
u = TrialFunction(V)
v = TestFunction(V)
f = Constant(0.1)
a = dot(grad(u), grad(v))*dx
L = f*v*dx

# Define boundary condition values
u0 = Constant(0.0)
u1 = Constant(1.0)
u2 = Constant(2.0)
u3 = Constant(3.0)

# Define boundary conditions
bc0 = DirichletBC(V, u0, 0)
bc1 = DirichletBC(V, u1, 1)
bc2 = DirichletBC(V, u2, 2)
bc3 = DirichletBC(V, u3, 3)

# Set PETSc MUMPS paramter (this is required to prevent a memory error
# in some cases when using MUMPS LU solver).
if has_petsc():
    PETScOptions.set("mat_mumps_icntl_14", 40.0)

# Compute solution
u = Function(V)
solve(a == L, u, [bc0, bc1, bc2, bc3])

# Write solution to file
File("u.pvd") << u
HTML(X3DOM().html(u))